In [27]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
model = Sequential(
[
    keras.Input((128,128,3)),
    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(16, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(8, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(4, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(1, activation='sigmoid')
]
)
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        25632     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0

In [45]:
@tf.function
def test_step(x,y):
    val_preds = model(x, training=False)
    # Update val metrics
    val_acc_metric.update_state(y, val_preds)
    return val_preds

In [46]:
model.load_weights('finalModel1.h5')

In [59]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'imgs/',
    target_size=(128, 128),
    batch_size=13,
    shuffle=True,
    class_mode='binary')

recent_test_batch = test_generator.next()
x_test_batch = recent_test_batch[0]
y_test_batch = recent_test_batch[1]

pred = test_step(x_test_batch,y_test_batch)
print(val_acc_metric.result())
for i in range(13):
    print(y_test_batch[i])
    print(np.reshape(tf.get_static_value(pred),(1,13))[0][i])


Found 13 images belonging to 2 classes.
tf.Tensor(0.6923076, shape=(), dtype=float32)
1.0
0.72792304
0.0
0.99154973
1.0
0.0009973347
0.0
0.0021025836
1.0
0.99886745
1.0
0.9998833
0.0
0.9694946
1.0
4.25559e-06
1.0
0.6748849
0.0
0.025391072
0.0
5.5991023e-12
0.0
0.00013306737
1.0
0.99778706
